In [3]:
#importing stuff

import cv2
import numpy as np
from collections import deque

ModuleNotFoundError: No module named 'training_fashion_mnist_dataset_practice'

In [31]:
#defining some stuff

#define lower_red and upper_red
redLower = np.array([170,100,60])
redUpper = np.array([180,255,255])

#define kernel
kernel = np.ones((5,5), np.uint8)

#points
points = deque(maxlen=512)

#get blackboard
blackboard = np.zeros((480, 640, 3), dtype = np.uint8)

counter = 0

In [33]:
#starting video

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1) #to flip the video
    
    #creating hsv and gray
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #detecting which pixels falls under our category
    red = cv2.inRange(hsv, redLower, redUpper)
    
    #processing the videos
    red = cv2.erode(red, kernel) #erroded
    red = cv2.morphologyEx(red, cv2.MORPH_OPEN, kernel) #opening
    red = cv2.dilate(red, kernel) #dilation
    
    #playing video
    
    cv2.imshow('red', red)
    
    #find contour
    _, contours, heirarchy = cv2.findContours(red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    draw_contours = cv2.cvtColor(red, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(draw_contours, contours, -1, (0,255,0), 3)
    cv2.imshow('draw_contours', draw_contours)
    
    centre = None
    
    #if any contour is found
    if len(contours) > 0:
        contour = sorted(contours, key = cv2.contourArea, reverse = True)[0] #contour = largest contour among all the contours found
        ((x,y),radius) = cv2.minEnclosingCircle(contour)
        cv2.circle(frame, (int(x), int(y)), int(radius), (125,344,278), 2)
        
        M = cv2.moments(contour) #M is a dict with 24 keys including m00,m10,m01.
        centres = (int(M['m10']/M['m00']), int(M['m01']/M['m00'])) #centres = centroid of given contour.
        points.appendleft(centre)
        
    #if no contours were found then it means that there is no red colour obj in surrounding
        
    elif len(contours) == 0:
        
        if len(points) != 0:
            blackboard_gray = cv2.cvtcolor(blackboard, cv2.COLOR_BGR2GRAY)
            blur = cv2.medianBlur(blackboard_gray, 15)
            blur = cv2.GaussianBlur(blur, (5,5), 0)
            thresh = cv2.thershold(blur, 0, 255, cv2.THERSH_BINARY + cv2.THRESH_OTSU)[1]
            cv2.imshow('thresh', thresh)
            blackboard_contours  = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
            
            #debug
            bb = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
            cv2.drawContours(bb, blackboard_contours, -1, (0,255,0), 3)
            cv2.imshow('bb', bb)
            
            if len(blackboard_contours) >= 1:
                contour = sorted(blackboard_contours, key = cv2.contourArea, reverse = True)[0]
                
                if cv2.conourArea(contour) > 1000:
                    x, y, w, h = cv2.boundingRect(contour)
                    alphabet = blackboard_gray[y-10:y+h+10, x-10:x+w+10]
                    
                    try:
                        img = cv2.resize(alphabet, (28,28))
                        cv2.imshow('alphabet', alphabet)
                        
                    except cv2.error as e:
                        points = deque(maxlen = 512)
                        blackboard = np.zeros((480, 640, 3), dtype = np.uint8)
                        continue
                        
                    img = np.array(img)
                    img = img.astype('float32')/255
                    prediction = probability_model.predict(img.reshape(1,28,28))[0]
                    prediction = np.argmax(prediction)
                    
                    #sound stuff
                    #......
                    
            #empty deque and blackboard for next trail
            points = deque(maxlen = 512)
            blackboard = np.zeros((480, 640, 3), dtype = np.uint8)
            
    #connect the points with line and identify letter
    
    for i in range(1, len(points)):
        if points[i-1] is None or points[i-1] is None:
            continue
        cv2.line(frame, points[i-1], points[i], (0,0,0), 8)
        cv2.line(blackboard, points[i-1], points[i], (255,255,255), 8)
        
    cv2.putText(frame, f"prediction: {class_name[int(np.argmax(prediction))]}", (20,400), cv2.FRONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 2)
    cv2.imshow('frame', frame)
        
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'letters' is not defined

In [23]:
len(contours)

18

In [26]:
cv2.imshow('blackboard', blackboard)